### Part 2: Programming Problems

#### Problem 1 (Stochastic Gradient Descent)

#### 1. Import the dataset

Load the dataset `height_weight_genders.csv` using pandas, and convert it into a NumPy array with gender encoded as 1 for Female and 0 for Male.

In [11]:
import pandas as pd
import numpy as np
from time import time

df = pd.read_csv('height_weight_genders.csv')

# Encode gender as binary: Female = 1, Male = 0
df['gender'] = (df['Gender'] == 'Female').astype(int)

# Extract relevant columns
data = df[['gender', 'Height', 'Weight']].values

#### 2. Standardise the height column

Compute the mean $ \mu $ and standard deviation $ \sigma $ of the height values, and replace each height with its standardised value.


In [12]:
# Standardise height column
heights = data[:, 1]
mu = heights.mean()
sigma = heights.std()
data[:, 1] = (heights - mu) / sigma

#### 3. Build data matrix $Z$ and target vector $Y$

Construct the data matrix $Z$ with a first column of ones and a second column of standardised heights. Create the vector $Y$ containing the weight values.

In [13]:
# Build matrix Z and vector Y
Z = np.column_stack((np.ones(len(data)), data[:, 1]))
Y = data[:, 2]
N = len(Y)

#### 4. Objective function

Define the function $ f(x) = \frac{1}{2N} \| Y - Zx \|^2 $ that measures prediction error.

In [14]:
def objective(Y, Z, x):
    """Compute prediction error f(x)."""
    residuals = Z.dot(x) - Y
    return 0.5 * np.dot(residuals, residuals) / N

#### 5. Gradient Function

We want to compute the gradient of the objective function  
$$
f(x) = \frac{1}{2N} \|Y - Zx\|^2.
$$

The gradient is given by  
$$
\nabla f(x) = \frac{1}{N} Z^T (Zx - Y),
$$  
which is the vectorized form of the average of all sample-wise gradients:  
$$
\nabla f(x) = \frac{1}{N} \sum_{i=1}^{N} \nabla f_i(x),
$$  
where  
$$
\nabla f_i(x) = (x^T z^{(i)} - y^{(i)}) z^{(i)}.
$$
We now implement this gradient function.


In [17]:
def gradient(Y, Z, x):
    """Compute gradient ∇f(x)."""
    return Z.T.dot(Z.dot(x) - Y) / N

#### 6. Strong convexity constant $ \mu $

Since $ \nabla^2 f(x) = \frac{1}{N} Z^T Z $, we compute the smallest eigenvalue of this matrix to estimate the strong convexity constant $ \mu $.

In [18]:
# Compute strong convexity constant
eigvals = np.linalg.eigh(Z.T.dot(Z) / N)[0]
mu = eigvals.min()
print(f"Strong convexity constant μ = {mu:.6f}")

Strong convexity constant μ = 1.000000


#### 7. Compute the closed-form solution

Solve the normal equations $ Z^T Z x^* = Z^T Y $ and evaluate $ f(x^*) $.

In [19]:
# Solve for x*
x_star = np.linalg.solve(Z.T @ Z, Z.T @ Y)
f_star = objective(Y, Z, x_star)

print("x* =", x_star)
print("f(x*) =", f_star)

x* = [161.44035683  29.69099655]
f(x*) = 74.64674197456483


#### 8. Evaluate one stochastic gradient

At $ x^{(0)} = (10, 2) $, select a random index $ i $ and compute one stochastic gradient $ \nabla f_i(x^{(0)}) $.

In [20]:
x0 = np.array([10.0, 2.0])
i = np.random.randint(N)
zi = Z[i]
yi = Y[i]

# Gradient for individual sample i
g0 = (zi @ x0 - yi) * zi
print("Stochastic gradient ∇f_i(x⁰) =", g0)

Stochastic gradient ∇f_i(x⁰) = [-165.79356381  -92.12795397]


#### 9. Verify unbiasedness empirically

Sample $ \nabla f_i(x^{(0)}) $ 10,000 times and average to compare with the full gradient $ \nabla f(x^{(0)}) $.

In [21]:
# Estimate expectation of stochastic gradient
g_avg = np.zeros(2)
for _ in range(10000):
    i = np.random.randint(N)
    zi, yi = Z[i], Y[i]
    g_avg += (zi @ x0 - yi) * zi
g_avg /= 10000

# Full gradient
full_grad = gradient(Y, Z, x0)

print("Average stochastic gradient:", g_avg)
print("Full gradient:", full_grad)

Average stochastic gradient: [-150.96828955  -25.14210348]
Full gradient: [-151.44035683  -27.69099655]


#### 10. Implement stochastic gradient for mini-batch

Write a function to compute a stochastic gradient using a mini-batch of size $ m $, sampled without replacement.

In [22]:
def stochastic_gradient(Y, Z, x, m):
    """Mini-batch stochastic gradient using batch size m (no replacement)."""
    idx = np.random.choice(N, m, replace=False)
    Z_batch = Z[idx]
    Y_batch = Y[idx]
    return Z_batch.T @ (Z_batch @ x - Y_batch) / m